In [1]:
import os
import re
import random
import shutil
from typing import Tuple, List
from PIL import Image

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from yolo_utils import *

%matplotlib inline

IMAGES_PATH = "/kaggle/input/11-xhec/images"
LABELS_PATH = "/kaggle/working/chronsite_images/labels/train"

FORMATTED_IMAGES_PATH = "/kaggle/working/chronsite_images_formatted/images/train"
FORMATTED_LABELS_PATH = "/kaggle/working/chronsite_images_formatted/labels/train"

VAL_IMAGES_PATH = "/kaggle/working/chronsite_images_formatted/images/val"

TEST_IMAGES_PATH = "/kaggle/input/11-xhec/Detection_Test_Set_Img/Detection_Test_Set_Img"
FORMATTED_TEST_IMAGES_PATH = "/kaggle/working/Detection_Test_Set_Img"

CSV_LABELS_PATH = "/kaggle/input/11-xhec/bboxes_yolo.csv"
CONFIG_FILE_PATH = "test_chronsite.yaml"

IMAGE_DIMENSION = 1920
N_IMAGES = 0     # set to 0 for all images available
BATCH_SIZE = 2    # too big will result in memory error
EPOCHS = 30
YOLO_SIZE = "l"   # "s" for small, "m" for medium, "l" for large, "x" for extra-large
VAL_SET_SIZE = 0.1
CONF_THRES_PRED = 0.4
N_AUGMENTATIONS = 0   # Number of augmentation of test images

In [2]:
clone_yolo_repo()

make_arborescence(LABELS_PATH, FORMATTED_IMAGES_PATH, FORMATTED_LABELS_PATH, FORMATTED_TEST_IMAGES_PATH)

write_labels_to_files(CSV_LABELS_PATH)

format_images_and_labels(N_IMAGES, IMAGE_DIMENSION, IMAGES_PATH, 
                         FORMATTED_LABELS_PATH, FORMATTED_IMAGES_PATH)

create_val_set(VAL_SET_SIZE, FORMATTED_IMAGES_PATH, FORMATTED_LABELS_PATH)

augment_train_set(FORMATTED_IMAGES_PATH, FORMATTED_LABELS_PATH, N_AUGMENTATIONS)

!mv yolov5/* ./

!pip install -qr requirements.txt

write_yolo_train_config(CONFIG_FILE_PATH, FORMATTED_IMAGES_PATH)

!python train.py --img {IMAGE_DIMENSION} --batch {BATCH_SIZE} --epochs {EPOCHS} --data /kaggle/working/{CONFIG_FILE_PATH} --cfg /kaggle/working/models/yolov5{YOLO_SIZE}.yaml --name yolov5{YOLO_SIZE}_fold0 --weights yolov5{YOLO_SIZE}.pt

Yolo repo cloned

Created folders organization

Labels written from csv to .txt files



1581 images and labels written to disk
0 images escaped because of Albumentations' bug

Created validation folders

Validation set created with 10% of initial images!
0 images not augmented because of bug
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

hypertools 0.6.2 requires scikit-learn<0.22,>=0.19.1, but you'll have scikit-learn 0.23.2 which is incompatible.
Config written: 
 
    # train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
    train: /kaggle/working/chronsite_images_formatted/images/train
    val: /kaggle/working/chronsite_images_formatted/images/val

    # number of classes
    nc: 2

    # class names
    names: ['People', 'Mixer_truck']
     

2020-10-06 16:3

In [3]:
weights_folder = [p for p in os.listdir("/kaggle/working/runs") if re.match(r"exp\d", p)][-1]
weights_path = f"./runs/{weights_folder}/weights/best.pt"

print(weights_path)

./runs/exp0_yolov5l_fold0/weights/best.pt


In [4]:
format_test_images(TEST_IMAGES_PATH, FORMATTED_TEST_IMAGES_PATH, IMAGE_DIMENSION)

#/kaggle/working/chronsite_images_formatted/images/val

! python detect.py --img-size {IMAGE_DIMENSION} --save-txt --source /kaggle/working/Detection_Test_Set_Img --weights ./runs/exp0_yolov5l_fold0/weights/best.pt --conf 0.01


Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.01, device='', img_size=1920, iou_thres=0.5, output='inference/output', save_txt=True, source='/kaggle/working/Detection_Test_Set_Img', update=False, view_img=False, weights=['./runs/exp0_yolov5l_fold0/weights/best.pt'])
Fusing layers... 
image 1/100 /kaggle/working/Detection_Test_Set_Img/2019-10-17-13-30-11.jpg: 1920x1920 1 Peoples, Done. (0.154s)
image 2/100 /kaggle/working/Detection_Test_Set_Img/2019-10-17-13-42-19.jpg: 1920x1920 7 Peoples, Done. (0.153s)
image 3/100 /kaggle/working/Detection_Test_Set_Img/2019-10-17-13-54-13.jpg: 1920x1920 8 Peoples, Done. (0.153s)
image 4/100 /kaggle/working/Detection_Test_Set_Img/2019-10-17-14-06-24.jpg: 1920x1920 10 Peoples, 1 Mixer_trucks, Done. (0.153s)
image 5/100 /kaggle/working/Detection_Test_Set_Img/2019-10-17-14-18-46.jpg: 1920x1920 11 Peoples, 3 Mixer_trucks, Done. (0.153s)
image 6/100 /kaggle/working/Detection_Test_Set_Img/2019-10-17-14-30-46.jpg: 1920x1920 15 Peopl

In [5]:
#os.listdir("/kaggle/working/inference/output")

In [6]:
def gather_predictions(txt_output_path: str) -> pd.DataFrame:
    text_files_paths = [p for p in os.listdir(txt_output_path) if ".txt" in p]
    
    total_objects_detected = dict()
    for text_file_path in text_files_paths:
        with open(os.path.join(txt_output_path, text_file_path)) as f:
            for i, line in enumerate(f.read().split("\n")):
                if line:
                    total_objects_detected[f"{text_file_path}_{i}"] = line.split()
                    
    df_objects_detected = pd.DataFrame(total_objects_detected).T
    
    col_types = {"category": int, "x_center": float, "y_center": float, "width": float, "height": float}
    df_objects_detected.columns = list(col_types.keys())
    df_objects_detected = df_objects_detected.astype(col_types)

    return df_objects_detected

In [7]:
df_objects_pred = gather_predictions("/kaggle/working/inference/output")

df_objects_pred.to_csv("predictions_yolonet.csv")

In [8]:
!rm -rf /kaggle/working/inference/output
!rm -rf /kaggle/working/utils
!rm -rf /kaggle/working/yolov5
!rm -rf /kaggle/working/chronsite_images
!rm -rf /kaggle/working/Detection_Test_Set_Img

In [9]:
from PIL import Image, ImageFont, ImageDraw, ImageEnhance

def from_yolo_to_cor(box, shape):
    img_h, img_w, _ = shape
    x1, y1 = int((box[0] + box[2]/2)*img_w), int((box[1] + box[3]/2)*img_h)
    x2, y2 = int((box[0] - box[2]/2)*img_w), int((box[1] - box[3]/2)*img_h)
    return x1, y1, x2, y2

def draw_boxes(img, boxes, categories, mapping):
    draw = ImageDraw.Draw(img)
    
    for box, categ in zip(boxes, categories):
        x1, y1, x2, y2 = from_yolo_to_cor(box, np.asarray(img).shape)
        draw.rectangle((x1, y1, x2, y2), outline="green")
        draw.text((x1, y1), f"GT: {mapping[categ]}")

    display(img)


In [10]:
VAL_IMAGES_TO_SHOW = 5
MAPPING = {0: "People", 1: "Mixter Truck"}

for _ in range(VAL_IMAGES_TO_SHOW):

    random_infered_image = random.choice([p for p in os.listdir("inference/output") if ".jpg" in p])
    #print(random_infered_image)

    im = Image.open(f"inference/output/{random_infered_image}").resize((700, 700))
"""
    with open("/kaggle/working/chronsite_images_formatted/labels/val/" + random_infered_image.split(".")[0] + ".txt") as f:
        annot = f.read()
        bboxes = [list(map(float, s.split()[1:])) for s in f.read().split("\n") if s]
        categories = [list(map(float, s.split()[0]))[0] for s in annot.split("\n") if s]

    draw_boxes(im, bboxes, categories, MAPPING)"""

FileNotFoundError: [Errno 2] No such file or directory: 'inference/output'